# Episodic Memory in Customer Support: Personalized Service Demo

This notebook demonstrates how **Episodic Memory** creates deeply personalized customer experiences by remembering each customer's unique history, circumstances, and past interactions.

## Core Concept: Every Customer Has a Unique Story

Episodic memory ensures that specific customer events, circumstances, and history are remembered across all interactions. Unlike semantic memory which stores general knowledge, episodic memory is personal - it remembers that John's mother has medical equipment, that Sarah's son just got his license, or that Michael had a traumatic water damage experience.

**Key Benefits:**
- **Personal Recognition**: Customers feel valued when agents remember their specific situations
- **Context-Aware Service**: Responses account for past events and customer circumstances  
- **Relationship Building**: Each interaction builds on the customer's unique journey
- **Proactive Support**: Agents can anticipate needs based on customer history

This demo shows how an insurance agent remembers a customer's previous water damage crisis involving medical equipment and provides personalized service months later when they call about a new issue.

In [ ]:
import asyncio
from typing import Dict, List, Any, TypedDict
import json
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_aws import ChatBedrockConverse
from mem0 import MemoryClient
import os

## Memory Client

The MemoryClient for episodic memory uses `user_id` to create individual memory spaces for each customer. Unlike semantic memory which is shared across all interactions, episodic memory is personal - each customer's history, circumstances, and experiences are stored separately and retrieved specifically for them.

In [ ]:
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("MEM0_API_KEY")

In [ ]:
import os
from mem0 import MemoryClient

memory_client = MemoryClient()

## LLM

Configured with low temperature for consistent, empathetic responses that appropriately reference customer history and demonstrate personal recognition in insurance support scenarios.

In [ ]:
llm = ChatBedrockConverse(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    temperature=0.1,
)

## Customer State for Personalized Interactions

The `CustomerState` tracks individual customer information and their personal history. The key difference from other memory types is the `is_returning_customer` flag and `customer_history` - enabling agents to provide personalized service based on each customer's unique journey and past experiences.

In [4]:
class CustomerState(TypedDict):
    """State for episodic memory demo"""
    customer_id: str
    customer_query: str
    customer_history: List[Dict[str, Any]]
    support_response: str
    is_returning_customer: bool

## Personal History Retrieval

This function searches for memories specific to an individual customer using their `user_id`. Unlike semantic memory which is shared across all customers, episodic memory retrieval is personal - it finds this specific customer's past interactions, life events, family circumstances, and unique situations.

**The Power of Personal Context:**
- Remembers individual family situations (elderly mother with medical equipment)
- Tracks customer tenure and loyalty history  
- Recalls past claims and their specific circumstances
- Identifies patterns unique to this customer's journey

This creates the foundation for truly personalized service where customers feel recognized and valued as individuals.

In [5]:
async def retrieve_customer_history(state: CustomerState) -> CustomerState:
    """Retrieve specific customer's episodic memories"""
    
    # Search for this specific customer's past interactions
    customer_memories = memory_client.search(
        query=state["customer_query"],
        user_id=state["customer_id"],
        limit=10
    )
    
    state["customer_history"] = customer_memories
    state["is_returning_customer"] = len(customer_memories) > 0
    return state

## Personalized Response Generation

Generates responses that reference the customer's specific history and circumstances. For returning customers, acknowledges past events and builds on previous interactions. For new customers, provides standard service while building their memory profile.

In [6]:
async def generate_personalized_response(state: CustomerState) -> CustomerState:
    """Generate response using customer's episodic memory"""
    
    # Extract customer's specific history
    history_context = []
    for memory in state["customer_history"]:
        content = memory.get('memory', '')
        if content:
            history_context.append(content)
    
    customer_context = "\n".join(history_context) if history_context else "No previous interactions found"
    
    # Generate response using episodic memory
    prompt = f"""
You are an insurance support agent. Use the customer's specific history to provide personalized service.

Current Query: {state["customer_query"]}

Customer's History:
{customer_context}

Is Returning Customer: {state["is_returning_customer"]}

If this is a returning customer:
- Reference their specific past events and circumstances
- Show you remember their unique situation
- Build on previous interactions
- Acknowledge their history with us

If this is a new customer:
- Provide helpful service but note you don't have their history
- Focus on gathering necessary information

Provide a personalized response that shows you remember them.
"""

    response = await llm.ainvoke(prompt)
    state["support_response"] = response.content.strip()
    
    return state

## Response Without Memory (Baseline)

Generates generic responses without access to customer history. Treats every interaction as if it's the first time meeting the customer, demonstrating the difference episodic memory makes.

In [7]:
async def generate_response_without_memory(query: str) -> str:
    """Generate response WITHOUT episodic memory - treating as new customer"""
    
    prompt = f"""
You are an insurance support agent. You have NO access to customer history or previous interactions.
Treat this as a first-time caller.

Customer Query: {query}

Provide a generic response as if you've never spoken to this customer before.
"""

    response = await llm.ainvoke(prompt)
    return response.content.strip()

## Storing Customer Interactions

Saves each interaction to the customer's personal memory space, building their individual history over time for future personalized service.

In [ ]:
async def store_interaction(state: CustomerState) -> CustomerState:
    """Store this interaction as episodic memory"""
    
    # Store the current interaction for future reference
    interaction = f"Customer called about: {state['customer_query']}. Resolution: {state['support_response']}"
    
    memory_client.add(
        messages=[{"role": "user", "content": state['customer_query']}, 
                  {"role": "assistant", "content": state['support_response']}],
        user_id=state["customer_id"],
        metadata={
            "type": "episodic_memory"
        }
    )
    
    return state

## Episodic Memory Workflow

Creates a workflow that retrieves customer history, generates personalized responses, and stores new interactions for future use.

In [9]:
def create_episodic_workflow() -> StateGraph:
    """Create workflow for episodic memory demo"""
    
    workflow = StateGraph(CustomerState)
    
    # Add nodes
    workflow.add_node("retrieve_customer_history", retrieve_customer_history)
    workflow.add_node("generate_personalized_response", generate_personalized_response)
    workflow.add_node("store_interaction", store_interaction)
    
    # Define flow
    workflow.set_entry_point("retrieve_customer_history")
    workflow.add_edge("retrieve_customer_history", "generate_personalized_response")
    workflow.add_edge("generate_personalized_response", "store_interaction")
    workflow.add_edge("store_interaction", END)
    
    # Compile
    memory_saver = MemorySaver()
    return workflow.compile(checkpointer=memory_saver)

## Workflow Compilation

Compiles the episodic memory workflow for processing customer queries with personalized context.

In [10]:
graph = create_episodic_workflow()

## Sample Customer Histories

Creates realistic customer histories including a water damage emergency with medical equipment concerns and a teen driver addition scenario. These personal stories become the foundation for personalized future interactions.

In [11]:
def store_customer_histories():
    """Store episodic memory examples - specific customer events"""
    
    # Customer 1: Water damage with elderly mother's medical equipment
    customer1_history = {
        "conversation_id": "EPI-001",
        "customer_id": "CUST-45678",
        "timestamp": "2024-01-15T09:00:00Z",
        "metadata": {
            "type": "urgent_claim",
            "insurance_type": "home"
        },
        "conversation": [
            {
                "role": "user",
                "content": "I have a major problem. A pipe burst in our basement and there's water everywhere. My elderly mother lives with us and her medical equipment is down there."
            },
            {
                "role": "assistant",
                "content": "This is definitely urgent. First, have you shut off the main water valve?"
            },
            {
                "role": "user",
                "content": "Yes, my husband did that. But my mother's oxygen concentrator, her CPAP machine, and her electric wheelchair are all in standing water. She has severe COPD and can't be without oxygen for long."
            },
            {
                "role": "assistant",
                "content": "I understand the critical nature. I'm immediately dispatching ServePro for emergency water mitigation - they'll be there within 2 hours. What's your mother's oxygen situation right now?"
            },
            {
                "role": "user",
                "content": "We have a portable tank but it only lasts 4 hours. The medical supply company can't deliver a replacement concentrator until tomorrow."
            },
            {
                "role": "assistant",
                "content": "I'm authorizing an emergency equipment rental. Call Brookdale Medical Supplies at 555-MEDICAL - tell them claim CLM-WATER-8899 pre-authorizes immediate rental of an oxygen concentrator, CPAP, and wheelchair. They'll bill us directly. Also, that basement bedroom will need special remediation due to medical use - I'm noting that for the restoration team."
            },
            {
                "role": "user",
                "content": "Thank you so much. We've been customers for 12 years and this is our first major claim. We're just overwhelmed."
            },
            {
                "role": "assistant",
                "content": "You're in good hands. Your loyalty is appreciated, and we're here for situations exactly like this. The restoration team will also set up air purifiers specifically rated for medical spaces. Keep all receipts for your mother's temporary relocation if needed - that's covered under additional living expenses."
            }
        ]
    }
    
    # Store the conversation
    memory_client.add(
        messages=customer1_history["conversation"],
        user_id=customer1_history["customer_id"],
        metadata=customer1_history["metadata"]
    )
    
    # Customer 2: Teen driver addition with specific details
    customer2_history = {
        "conversation_id": "PRO-001",
        "customer_id": "CUST-56789",
        "timestamp": "2024-02-01T11:00:00Z",
        "metadata": {
            "type": "policy_change",
            "insurance_type": "auto"
        },
        "conversation": [
            {
                "role": "user",
                "content": "My son Michael just turned 16 and got his license. I'm dreading this - I heard adding a teen can double our premium. He's a good kid, 3.8 GPA, finished driver's ed last month."
            },
            {
                "role": "assistant",
                "content": "Congratulations to Michael! And don't worry - with his qualifications, the impact will be much less than doubling. Let me walk you through how to minimize the cost. First, that 3.8 GPA qualifies for our good student discount of 15%. The driver's education certificate adds another 10%. Now, here's the key: which car will he primarily drive?"
            },
            {
                "role": "user",
                "content": "We have a 2022 BMW X5, a 2020 Tesla Model 3, and a 2015 Honda Civic. He likes the Tesla but we were thinking the Honda would be safer for him to learn on."
            },
            {
                "role": "assistant",
                "content": "Perfect thinking! Assigning Michael as primary driver on the 2015 Civic will save you hundreds per month versus the Tesla or BMW. I'm applying all discounts and enrolling him in TeenSafe monitoring for another 20% off. Your total increase will be about $180/month instead of the $400+ you were fearing."
            }
        ]
    }
    
    memory_client.add(
        messages=customer2_history["conversation"],
        user_id=customer2_history["customer_id"],
        metadata=customer2_history["metadata"]
    )

In [ ]:
print("\n\n📚 [Loading customer histories into episodic memory...]")
store_customer_histories()
print("✅ Customer histories loaded!\n")

## Customer Query Handler

Processes customer queries through the episodic memory workflow, returning personalized responses based on their individual history and circumstances.

In [12]:
async def handle_customer_query(customer_id: str, query: str) -> Dict[str, Any]:
    """Process a customer query using episodic memory"""
    
    initial_state = CustomerState(
        customer_id=customer_id,
        customer_query=query,
        customer_history=[],
        support_response="",
        is_returning_customer=False
    )
    
    config = {"configurable": {"thread_id": f"episodic_{customer_id}"}}
    result = await graph.ainvoke(initial_state, config)
    
    return {
        "query": query,
        "response": result["support_response"],
        "history_items_found": len(result["customer_history"]),
        "is_returning": result["is_returning_customer"]
    }

## Episodic Memory Demonstration

The following cells demonstrate the power of episodic memory by comparing responses with and without customer history. The same customer who previously experienced a traumatic water damage incident now calls about a new water issue. Notice how episodic memory enables empathetic, context-aware service.

In [ ]:
print("💧 SCENARIO 1: Water Damage Customer - 6 Months Later")
print("=" * 80)

# First, simulate WITHOUT episodic memory
print("\n❌ WITHOUT EPISODIC MEMORY (Generic Agent):")
print("-" * 60)
query1 = "Hi, I need help. We're having another water issue, this time from the water heater."
response_no_memory1 = await generate_response_without_memory(query1)
print(f"Customer (CUST-45678): {query1}")
print(f"\nAgent: {response_no_memory1}")


In [15]:


# With episodic memory
print("\n✅ WITH EPISODIC MEMORY (Personalized Agent):")
print("-" * 60)
result1 = await handle_customer_query(
    customer_id="CUST-45678",
    query=query1
)
print(f"Customer (CUST-45678): {query1}")
print(f"\nAgent: {result1['response']}")
print(f"\n✨ Customer history items found: {result1['history_items_found']}")
print(f"✨ Recognized as returning customer: {result1['is_returning']}")


✅ WITH EPISODIC MEMORY (Personalized Agent):
------------------------------------------------------------
Customer (CUST-45678): Hi, I need help. We're having another water issue, this time from the water heater.

Agent: Hi there. I see we're dealing with another water-related incident, and I want you to know I'm here to help you through this, especially given what you've already been through with the previous pipe burst.

I remember the significant challenges you faced last time with your mother's critical medical equipment, and I understand how stressful this must be. Given your mother's severe COPD and her dependence on medical devices, we need to act quickly to mitigate the water damage and ensure her safety.

Here's what I recommend we do immediately:
1. I'm escalating this claim to our emergency water damage response team
2. We'll prioritize getting professional water removal and drying services to your home today
3. I'll help coordinate with your medical equipment providers to 

In [16]:
print("\n\n💧 SCENARIO 1B: Follow-up Question")
print("=" * 80)

query1b = "Yes, thankfully it's in a different area. But I'm worried about repeated water claims affecting our rates."

print("\n❌ WITHOUT EPISODIC MEMORY:")
print("-" * 60)
response_no_memory1b = await generate_response_without_memory(query1b)
print(f"Customer: {query1b}")
print(f"\nAgent: {response_no_memory1b}")
print("\n⚠️  Notice: Can't provide context-aware advice about their specific situation")




💧 SCENARIO 1B: Follow-up Question

❌ WITHOUT EPISODIC MEMORY:
------------------------------------------------------------
Customer: Yes, thankfully it's in a different area. But I'm worried about repeated water claims affecting our rates.

Agent: I apologize, but I'm having trouble understanding the context of your statement. It seems like you're referring to a previous conversation about water damage or insurance claims, but I don't have any prior information. Could you please provide me with more details about your specific situation? I'd be happy to help you understand how water claims might impact your insurance rates.

⚠️  Notice: Can't provide context-aware advice about their specific situation


In [ ]:
print("\n\n✅ WITH EPISODIC MEMORY:")
print("-" * 60)
result1b = await handle_customer_query(
    customer_id="CUST-45678",
    query=query1b
)
print(f"Customer: {query1b}")
print(f"\nAgent: {result1b['response']}")